##IMPORTS

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import os
import numpy as np
import random
import h5py
import sys
from decimal import Decimal
import tensorflow as tf
from sklearn.model_selection import train_test_split
import keras
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras import backend as K
from keras.losses import categorical_crossentropy
from keras.metrics import categorical_accuracy
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.layers import UpSampling2D, Cropping2D
from keras import models

Using TensorFlow backend.


## Running this Notebook 
https://drive.google.com/drive/folders/1mf9UHHPq6tg8kZGlFTrFpCJfkg4zhWiB?usp=sharing

Can be open this with NYU account,

Please add this to your google Drive and you're good to go

You can find results you generated in "results" and "repaired-nets" folders
We stored results we got in "results_we_got" and "repaired-nets_we_got"
So if you open those you can directly verify the results we are able to obtain


**GET DATA FROM DRIVE**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
sys.path.append(os.path.abspath("/content/drive/MyDrive/MLSecurityProject/")) 


Mounted at /content/drive


## API PARAMETERS

Change the file names hear

In [ ]:
MODEL_DIR = '/content/drive/MyDrive/MLSecurityProject/models'  # model directory
MODEL_FILENAME = 'sunglasses_bd_net.h5'  # badnet model file

DATA_DIR = '/content/drive/MyDrive/MLSecurityProject/data'  # data folder
DATA_FILE = 'clean_validation_data.h5' # dataset file
POISONED_FILE = 'sunglasses_poisoned_data.h5'
TEST_FILE = 'clean_test_data.h5'

RESULT_DIR = '/content/drive/MyDrive/MLSecurityProject/results'  # directory for storing results
# image filename template for visualization results
IMG_FILENAME_TEMPLATE = 'visualize_%s_label_%d.png'
REPAIRED_NETS= '/content/drive/MyDrive/MLSecurityProject/repaired-networks'
REPAIRED_NET_FILENAME_TEMPLATE = 'repaired_net_%s.h5'

## DATA PROCESSING

In [ ]:
def save_image(x, file, frmt):
    img = image.array_to_img(x, scale=False)
    img.save(file, frmt)
    return

In [ ]:
def load_data(data_file, keys=None):
    data = {}
    with h5py.File(data_file, 'r') as hf:
        if keys is None:
            for n in hf:
                data[n] = np.array(hf.get(n))
        else:
            for n in keys:
                data[n] = np.array(hf.get(n))

    return data

In [ ]:
def unique_data(X_test, Y_test):
    Y_test_unique = np.unique(Y_test)
    return len(Y_test_unique)

In [ ]:
def ld_data(data=('%s/%s' % (DATA_DIR, DATA_FILE))):
    my_data = load_data(data, keys=['data', 'label'])
    X = np.array(my_data['data'], dtype='float32')
    Y = np.array(my_data['label'], dtype='float32')
    return X, Y

In [ ]:
def build_data_loader(X, Y):
    img_gen = ImageDataGenerator().flow(X, Y, batch_size=batchSize)
    return img_gen

## VISUALISER CLASS


In [ ]:
#### referred from https://github.com/bolunwang/backdoor
class Visualizer:
    
    # type of regularization of the mask
    REGULARIZATION = 'l1'
    # threshold of attack success rate for dynamically changing cost
    ATTACK_SUCC_THRESHOLD = 0.99
    # patience
    PATIENCE = 5
    # multiple of changing cost, down multiple is the square root of this
    COST_MULTIPLIER = 2 
    # if resetting cost to 0 at the beginning
    # default is true for full optimization, set to false for early detection
    RESET_COST_TO_ZERO = True
    # min/max of mask
    MASK_MIN = 0
    MASK_MAX = 1
    # min/max of raw pixel intensity
    COLOR_MIN = 0
    COLOR_MAX = 255
    # whether to shuffle during each epoch
    SHUFFLE = True
    # batch size of optimization
    BATCH_SIZE = 32
    # verbose level, 0, 1 or 2
    VERBOSE = 1
    # whether to return log or not
    RETURN_LOGS = True
    # whether to save last pattern or best pattern
    SAVE_LAST = False
    # epsilon used in tanh
    EPSILON = K.epsilon()
    # early stop flag
    EARLY_STOP = True
    # early stop threshold
    EARLY_STOP_THRESHOLD = 0.99
    # early stop patience
    EARLY_STOP_PATIENCE = 5 * PATIENCE
    # save tmp masks, for debugging purpose
    SAVE_TMP = False
    # dir to save intermediate masks
    TMP_DIR = 'tmp'
    # whether input image has been preprocessed or not
    RAW_INPUT_FLAG = False

    def __init__(self, model, regularization, input_shape, 
                 init_cost, steps, mini_batch, lr, num_classes,
                 attack_succ_threshold=ATTACK_SUCC_THRESHOLD,
                 patience=PATIENCE, cost_multiplier=COST_MULTIPLIER,
                 reset_cost_to_zero=RESET_COST_TO_ZERO,
                 mask_min=MASK_MIN, mask_max=MASK_MAX,
                 color_min=COLOR_MIN, color_max=COLOR_MAX, 
                 shuffle=SHUFFLE, batch_size=BATCH_SIZE, verbose=VERBOSE,
                 return_logs=RETURN_LOGS, save_last=SAVE_LAST,
                 epsilon=EPSILON,
                 early_stop=EARLY_STOP,
                 early_stop_threshold=EARLY_STOP_THRESHOLD,
                 early_stop_patience=EARLY_STOP_PATIENCE,
                 save_tmp=SAVE_TMP, tmp_dir=TMP_DIR):

        self.model = model
        self.regularization = regularization
        self.input_shape = input_shape
        self.init_cost = init_cost
        self.steps = steps
        self.mini_batch = mini_batch
        self.lr = lr
        self.num_classes = totalClasses
        self.attack_succ_threshold = attack_succ_threshold
        self.patience = patience
        self.cost_multiplier_up = cost_multiplier
        self.cost_multiplier_down = cost_multiplier ** 1.5
        self.reset_cost_to_zero = reset_cost_to_zero
        self.mask_min = mask_min
        self.mask_max = mask_max
        self.color_min = color_min
        self.color_max = color_max
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.verbose = verbose
        self.return_logs = return_logs
        self.save_last = save_last
        self.epsilon = epsilon
        self.early_stop = early_stop
        self.early_stop_threshold = early_stop_threshold
        self.early_stop_patience = early_stop_patience
        self.save_tmp = save_tmp
        self.tmp_dir = tmp_dir

        mask_size =np.array(input_shape[0:2], dtype=float) 
        mask_size = mask_size.astype(int)
        self.mask_size = mask_size
        mask = np.zeros(self.mask_size)
        pattern = np.zeros(input_shape)
        mask = np.expand_dims(mask, axis=2)

        mask_tanh = np.zeros_like(mask)
        pattern_tanh = np.zeros_like(pattern)

        # prepare mask related tensors
        self.mask_tanh_tensor = K.variable(mask_tanh)
        mask_tensor_unrepeat = (K.tanh(self.mask_tanh_tensor) /
                                (2 - self.epsilon) +
                                0.5)
        mask_tensor_unexpand = K.repeat_elements(
            mask_tensor_unrepeat,
            rep=3, 
            axis=2)
        self.mask_tensor = K.expand_dims(mask_tensor_unexpand, axis=0)
        upsample_layer = UpSampling2D(size=(1, 1))
        mask_upsample_tensor_uncrop = upsample_layer(self.mask_tensor)
        uncrop_shape = K.int_shape(mask_upsample_tensor_uncrop)[1:]
        cropping_layer = Cropping2D(
            cropping=((0, uncrop_shape[0] - self.input_shape[0]),
                      (0, uncrop_shape[1] - self.input_shape[1])))
        self.mask_upsample_tensor = cropping_layer(
            mask_upsample_tensor_uncrop)
        reverse_mask_tensor = (K.ones_like(self.mask_upsample_tensor) -
                               self.mask_upsample_tensor)

        # prepare pattern related tensors
        self.pattern_tanh_tensor = K.variable(pattern_tanh)
        self.pattern_raw_tensor = (
            (K.tanh(self.pattern_tanh_tensor) / (2 - self.epsilon) + 0.5) *
            255.0)

        # prepare input image related tensors
        # ignore clip operation here
        # assume input image is already clipped into valid color range
        input_tensor = K.placeholder(model.input_shape)
        input_raw_tensor = input_tensor * 255.0

        # IMPORTANT: MASK OPERATION IN RAW DOMAIN
        X_adv_raw_tensor = (
            reverse_mask_tensor * input_raw_tensor +
            self.mask_upsample_tensor * self.pattern_raw_tensor)

        X_adv_tensor = X_adv_raw_tensor/255.0

        output_tensor = model(X_adv_tensor)
        y_true_tensor = K.placeholder(model.output_shape)

        self.loss_acc = categorical_accuracy(output_tensor, y_true_tensor)

        self.loss_ce = categorical_crossentropy(output_tensor, y_true_tensor)

        if self.regularization is None:
            self.loss_reg = K.constant(0)
        elif self.regularization is 'l1':
            self.loss_reg = (K.sum(K.abs(self.mask_upsample_tensor)) /3 )

        cost = self.init_cost
        self.cost_tensor = K.variable(cost)
        self.loss = self.loss_ce + self.loss_reg * self.cost_tensor

        self.opt = Adam(lr=self.lr, beta_1=0.5, beta_2=0.9)
        self.updates = self.opt.get_updates(
            params=[self.pattern_tanh_tensor, self.mask_tanh_tensor],
            loss=self.loss)
        self.train = K.function(
            [input_tensor, y_true_tensor],
            [self.loss_ce, self.loss_reg, self.loss, self.loss_acc],
            updates=self.updates)


    def reset_opt(self):

        K.set_value(self.opt.iterations, 0)
        for w in self.opt.weights:
            K.set_value(w, np.zeros(K.int_shape(w)))

    def reset_state(self, pattern_init, mask_init):

        print('resetting state')

        # setting cost
        if self.reset_cost_to_zero:
            self.cost = 0
        else:
            self.cost = self.init_cost
        K.set_value(self.cost_tensor, self.cost)

        # setting mask and pattern
        mask = np.array(mask_init)
        pattern = np.array(pattern_init)
        mask = np.clip(mask, self.mask_min, self.mask_max)
        pattern = np.clip(pattern, self.color_min, self.color_max)
        mask = np.expand_dims(mask, axis=2)

        # convert to tanh space
        mask_tanh = np.arctanh((mask - 0.5) * (2 - self.epsilon))
        pattern_tanh = np.arctanh((pattern / 255.0 - 0.5) * (2 - self.epsilon))
        print('mask_tanh', np.min(mask_tanh), np.max(mask_tanh))
        print('pattern_tanh', np.min(pattern_tanh), np.max(pattern_tanh))

        K.set_value(self.mask_tanh_tensor, mask_tanh)
        K.set_value(self.pattern_tanh_tensor, pattern_tanh)

        # resetting optimizer states
        self.reset_opt()


    def save_tmp_func(self, step):

        cur_mask = K.eval(self.mask_upsample_tensor)
        cur_mask = cur_mask[0, ..., 0]
        img_filename = ('%s/%s' % (self.tmp_dir, 'tmp_mask_step_%d.png' % step))
        save_image(np.expand_dims(cur_mask, axis=2) * 255,
                                  img_filename,
                                  'png')

        cur_fusion = K.eval(self.mask_upsample_tensor *
                            self.pattern_raw_tensor)
        cur_fusion = cur_fusion[0, ...]
        img_filename = ('%s/%s' % (self.tmp_dir, 'tmp_fusion_step_%d.png' % step))
        save_image(cur_fusion, img_filename, 'png')


    def visualize(self, gen, y, pattern_init, mask_init):

        # since we use a single optimizer repeatedly, we need to reset
        # optimzier's internal states before running the optimization
        self.reset_state(pattern_init, mask_init)

        # best optimization results
        mask_best = None
        mask_upsample_best = None
        pattern_best = None
        reg_best = float('inf')

        # logs and counters for adjusting balance cost
        logs = []
        cost_set_counter = 0
        cost_up_counter = 0
        cost_down_counter = 0
        cost_up_flag = False
        cost_down_flag = False

        # counter for early stop
        early_stop_counter = 0
        early_stop_reg_best = reg_best

        # vectorized target
        Y_ = to_categorical([y]* self.batch_size ,self.num_classes) 

        # loop start
        for step in range(self.steps):

            # record loss for all mini-batches
            loss_ce_list = []
            loss_reg_list = []
            loss_list = []
            loss_acc_list = []
            for idx in range(self.mini_batch):
                X_batch, _ = gen.next()
                if X_batch.shape[0] != Y_.shape[0]:
                    Y_ = to_categorical([y]* X_batch.shape[0],self.num_classes)
                (loss_ce_value,
                    loss_reg_value,
                    loss_value,
                    loss_acc_value) = self.train([X_batch, Y_])
                loss_ce_list.extend(list(loss_ce_value.flatten()))
                loss_reg_list.extend(list(loss_reg_value.flatten()))
                loss_list.extend(list(loss_value.flatten()))
                loss_acc_list.extend(list(loss_acc_value.flatten()))

            avg_loss_ce = np.mean(loss_ce_list)
            avg_loss_reg = np.mean(loss_reg_list)
            avg_loss = np.mean(loss_list)
            avg_loss_acc = np.mean(loss_acc_list)

            # check to save best mask or not
            if avg_loss_acc >= self.attack_succ_threshold and avg_loss_reg < reg_best:
                mask_best = K.eval(self.mask_tensor)
                mask_best = mask_best[0, ..., 0]
                mask_upsample_best = K.eval(self.mask_upsample_tensor)
                mask_upsample_best = mask_upsample_best[0, ..., 0]
                pattern_best = K.eval(self.pattern_raw_tensor)
                reg_best = avg_loss_reg

            # verbose
            if self.verbose != 0:
                if self.verbose == 2 or step % (self.steps // 10) == 0:
                    print('step: %3d, cost: %.2E, attack: %.3f, loss: %f, ce: %f, reg: %f, reg_best: %f' %
                          (step, Decimal(self.cost), avg_loss_acc, avg_loss,
                           avg_loss_ce, avg_loss_reg, reg_best))

            # save log
            logs.append((step,
                         avg_loss_ce, avg_loss_reg, avg_loss, avg_loss_acc,
                         reg_best, self.cost))

            # check early stop
            if self.early_stop:
                # only terminate if a valid attack has been found
                if reg_best < float('inf'):
                    if reg_best >= self.early_stop_threshold * early_stop_reg_best:
                        early_stop_counter += 1
                    else:
                        early_stop_counter = 0
                early_stop_reg_best = min(reg_best, early_stop_reg_best)

                if (cost_down_flag and
                        cost_up_flag and
                        early_stop_counter >= self.early_stop_patience):
                    print('early stop')
                    break

            # check cost modification
            if self.cost == 0 and avg_loss_acc >= self.attack_succ_threshold:
                cost_set_counter += 1
                if cost_set_counter >= self.patience:
                    self.cost = self.init_cost
                    K.set_value(self.cost_tensor, self.cost)
                    cost_up_counter = 0
                    cost_down_counter = 0
                    cost_up_flag = False
                    cost_down_flag = False
                    print('initialize cost to %.2E' % Decimal(self.cost))
            else:
                cost_set_counter = 0

            if avg_loss_acc >= self.attack_succ_threshold:
                cost_up_counter += 1
                cost_down_counter = 0
            else:
                cost_up_counter = 0
                cost_down_counter += 1

            if cost_up_counter >= self.patience:
                cost_up_counter = 0
                if self.verbose == 2:
                    print('up cost from %.2E to %.2E' %
                          (Decimal(self.cost),
                           Decimal(self.cost * self.cost_multiplier_up)))
                self.cost *= self.cost_multiplier_up
                K.set_value(self.cost_tensor, self.cost)
                cost_up_flag = True
            elif cost_down_counter >= self.patience:
                cost_down_counter = 0
                if self.verbose == 2:
                    print('down cost from %.2E to %.2E' %
                          (Decimal(self.cost),
                           Decimal(self.cost / self.cost_multiplier_down)))
                self.cost /= self.cost_multiplier_down
                K.set_value(self.cost_tensor, self.cost)
                cost_down_flag = True

            if self.save_tmp:
                self.save_tmp_func(step)

        # save the final version
        if mask_best is None or self.save_last:
            mask_best = K.eval(self.mask_tensor)
            mask_best = mask_best[0, ..., 0]
            mask_upsample_best = K.eval(self.mask_upsample_tensor)
            mask_upsample_best = mask_upsample_best[0, ..., 0]
            pattern_best = K.eval(self.pattern_raw_tensor)

        if self.return_logs:
            return pattern_best, mask_best, mask_upsample_best, logs
        else:
            return pattern_best, mask_best, mask_upsample_best

## GET POTENTIAL TRIGGERS

In [ ]:
def trigger_visualizer(visualizer, gen, y, pattern_flag=True):
    myPattern = np.random.random(inpShape) * 255.0
    myMask = np.random.random(maskShape)
    finalPattern, finalMask, finalMaskUpsample, logs = visualizer.visualize(
        gen=gen, y=y, pattern_init=myPattern, mask_init=myMask)
    if pattern_flag:
        save_pattern(finalPattern, finalMaskUpsample,y)

    return finalPattern, finalMaskUpsample, logs



In [ ]:
def save_pattern(pattern, mask, y):
    if not os.path.exists(RESULT_DIR):
        os.mkdir(RESULT_DIR)

    img_filename = ('%s/%s' % (RESULT_DIR,IMG_FILENAME_TEMPLATE % ('pattern', y)))
    save_image(pattern, img_filename, 'png')

    img_filename = ('%s/%s' % (RESULT_DIR,IMG_FILENAME_TEMPLATE % ('mask', y)))
    save_image(np.expand_dims(mask, axis=2) * 255,img_filename,'png')

    fusion = np.multiply(pattern, np.expand_dims(mask, axis=2))
    img_filename = ('%s/%s' % (RESULT_DIR,IMG_FILENAME_TEMPLATE % ('fusion', y)))
    save_image(fusion, img_filename, 'png')



In [ ]:
def get_potential_triggers():
    X_test, Y_test = ld_data()
    X_test=X_test.reshape(X_test.shape[0],X_test.shape[2],X_test.shape[3],X_test.shape[1])
    test_generator = build_data_loader(X_test, Y_test,)
    model_file = '%s/%s' % (MODEL_DIR, MODEL_FILENAME)
    model = load_model(model_file)
    visualizer_pi = Visualizer(model,regularization='l1',input_shape=inpShape,init_cost=initCost, steps=numSteps, lr=lr, num_classes=totalClasses,mini_batch=min_batch )
    logs_ = {}
    class_list = list(range(totalClasses))
    for cl in range(1241,totalClasses):

        print('processing label %d' % cl)

        pattern_best, _, logs = trigger_visualizer(
            visualizer_pi, test_generator, y=cl,
            pattern_flag=True)
        logs_[cl] = logs



## FILTER POTENTIAL TRIGGERS TO GET ACTUAL TRIGGERS

In [ ]:
def detect_outliers(normList, idx):
    consistConst = 1.4826  
    median = np.median(normList)
    mad = consistConst * np.median(np.abs(normList - median))
    minMad = np.abs(np.min(normList) - median)
    minMad = minMad/mad

    print('median: %f, MAD: %f' % (median, mad))
    print('anomaly index: %f' % minMad)

    flagList = []
    for cl in idx:
        if normList[idx[cl]] < median and np.abs(normList[idx[cl]] - median) / mad > 2:
            flagList.append((cl, normList[idx[cl]]))

    if len(flagList) > 0:
        flagList = sorted(flagList, key=lambda x: x[1])

    print('flagged list: %s' %', '.join(['%d: %2f' % (cl, norm)for cl, norm in flagList]))


In [ ]:
def analyze_pattern():
    maskFlatten = []
    idx = {}
    for y in range(totalClasses):
        maskFile = IMG_FILENAME_TEMPLATE % ('mask', y)
        if os.path.isfile('%s/%s' % (RESULT_DIR, maskFile)):
            img = image.load_img('%s/%s' % (RESULT_DIR, maskFile),color_mode='grayscale',target_size=inpShape)
            mask = image.img_to_array(img)
            mask = mask/255
            mask = mask[:, :, 0]

            maskFlatten.append(mask.flatten())

            idx[y] = len(maskFlatten) - 1

    normList = [np.sum(np.abs(m)) for m in maskFlatten]

    print('%d labels found' % len(normList))

    detect_outliers(normList, idx)

## Backdoor mitigation functions <br>


In [ ]:
def get_random_samples(X, Y, split):
    X_tr, X_clean,y_tr, y_clean= train_test_split(X, Y, train_size=split, random_state=42, shuffle=True)
    return X_tr,X_clean,y_tr, y_clean

In [ ]:
def get_mask_pattern(y):
    maskFile = IMG_FILENAME_TEMPLATE % ('mask', y)
    mask = None
    pattern = None
    img = image.load_img('%s/%s' % (RESULT_DIR, maskFile),target_size=inpShape)
    mask = image.img_to_array(img)

    patternFile = IMG_FILENAME_TEMPLATE % ('pattern', y)
    img = image.load_img('%s/%s' % (RESULT_DIR, patternFile),target_size=inpShape)
    pattern = image.img_to_array(img)
    return mask, pattern

def injection(mask, pattern, adv_img):
    return mask * pattern + (1 - mask) * adv_img

def infect_X(img, tgt):
    mask, pattern = get_mask_pattern(tgt)
    rawImg = np.copy(img)
    advImg = np.copy(rawImg)
    advImg = injection(mask, pattern, advImg)
    return advImg, tgt 

In [ ]:
def get_avg_neuron_activation():
    modelFile = '%s/%s' % (MODEL_DIR, MODEL_FILENAME)
    model_cl = load_model(modelFile)
    model_cl.fit(x=X_mtg, y=Y_mtg, epochs=5 ,shuffle=True)
    layer_outputs_cl = [layer.output for layer in model_cl.layers[12:13]] 
    activation_model_cl = models.Model(inputs=model_cl.input, outputs=layer_outputs_cl)

    model_ps = load_model(modelFile)
    model_ps.fit(x= patchImages, y=patchLabels, epochs=5 ,shuffle=True)
    layer_outputs_ps = [layer.output for layer in model_ps.layers[12:13]] 
    activation_model_ps = models.Model(inputs=model_ps.input, outputs=layer_outputs_ps)

    activations_cl=[]
    activations_cl=activation_model_cl.predict(X_mtg[:1000])
    avg_activation_cl = sum(np.array(activations_cl))/len(activations_cl) 

    activations_ps=[]
    activations_ps =activation_model_ps.predict(patchImages[:1000])
    avg_activation_ps = sum(np.array(activations_ps))/len(activations_ps) 
    diff_activation = avg_activation_ps - avg_activation_cl
    mean_act = sum(avg_activation_cl)/len(avg_activation_cl)
    return mean_act,diff_activation


In [ ]:
#Filter for Detecting Adversarial Inputs using Nueron Activations

def detect_adversial_inputs(model,x_i,mean_act):
    x=[]
    x.append(x_i)
    x=np.array(x)
    layer_outputs = [layer.output for layer in model.layers[12:13]] 
    activation_model = models.Model(inputs=model.input, outputs=layer_outputs)
    for neuron in activation_model.predict(x)[0] :
        if neuron > mean_act+1000 :
          return True
    return False



In [ ]:
#Neuron pruning by setting the neuron value to zero in last but 2nd layer when weight of the neuron > threshold
def neuron_pruning(model):
    modelFile = '%s/%s' % (MODEL_DIR, MODEL_FILENAME)
    new_model = load_model(modelFile)
    new_weights = model.get_weights().copy()
    for layer in new_weights[12]:
      for i in range(0,len(layer)):
          if layer[i] > mean_act+1000 :
              layer[i] =0
    return new_model

In [ ]:
def reverse_trigger_to_clean_images(X_mtg, Y_mtg):
    patchImages = []
    patchLabels = []
    imgIndex=[]
    for ind, img in enumerate(X_mtg):
        y = Y_mtg[ind]
        patchImg, patchLabel = infect_X(img, y)
        patchImages.append(patchImg)
        patchLabels.append(patchLabel)
        imgIndex.append(ind)

    patchImages = np.copy(patchImages)
    patchLabels = np.copy(patchLabels)
    return patchImages, patchLabels

##PARAMETERS 

set the optimization parameters here

In [ ]:
X, Y = ld_data()
xx=X.reshape(X.shape[0],X.shape[2],X.shape[3],X.shape[1])
inpShape = (xx.shape[1], xx.shape[2], xx.shape[3])
totalClasses = unique_data(X, Y) 
# parameters for optimization
batchSize= 10
lr = 0.1  # learning rate
numSteps = 100  # total optimization iterations 
num_sample_mb = 100  # number of samples in each mini batch
min_batch = num_sample_mb // batchSize  # mini batch size used for early stop
initCost = 1e-3  # initial weight used for balancing two objectives
maskShape = np.ceil(np.array(inpShape[0:2], dtype=float) )
maskShape = maskShape.astype(int)

## API

In [ ]:
repairedNetworkFile =REPAIRED_NETS+'/'+REPAIRED_NET_FILENAME_TEMPLATE % MODEL_FILENAME.split('.')[0]
X_tr,X_mtg,Y_tr,Y_mtg = get_random_samples(xx, Y, 0.7)
patchImages, patchLabels = reverse_trigger_to_clean_images(X_mtg, Y_mtg)
mean_act,_ = get_avg_neuron_activation()
def repaired_network():
    get_potential_triggers()
    analyze_pattern()

    #Neuron Pruning over the model
    modelFile = '%s/%s' % (MODEL_DIR, MODEL_FILENAME)
    model = load_model(modelFile)
    model.fit(x=X_tr, y=Y_tr, epochs=10 ,shuffle=True)
    model=neuron_pruning(model)
    model.save(repairedNetworkFile)
    model=load_model(repairedNetworkFile)

    # Evaluation over Neuron Pruned model with clean data
    X_test,Y_test=ld_data(data=('%s/%s' % (DATA_DIR, TEST_FILE)))
    X_test=X_test.reshape(X_test.shape[0],X_test.shape[2],X_test.shape[3],X_test.shape[1])
    _,accuracy = model.evaluate(x=X_test, y=Y_test)
    print("accuracy on clean data:"+ str(accuracy))

    # Evaluation over Neuron Pruned model with poisonous data
    X_ps , Y_ps =ld_data(data=('%s/%s' % (DATA_DIR, POISONED_FILE)))
    X_ps=X_ps.reshape(X_ps.shape[0],X_ps.shape[2],X_ps.shape[3],X_ps.shape[1])
    _,accuracy = model.evaluate(x=X_test, y=Y_test)
    print("accuracy on poisoned data:"+ str(accuracy))
    return model
    

In [ ]:
model =repaired_network()

12830/12830 [==============================] - 15s 1ms/step
accuracy on clean data:0.00046765393926762044
12830/12830 [==============================] - 14s 1ms/step
accuracy on poisoned data:0.00046765393926762044


In [ ]:
X_ps , Y_ps =ld_data(data=('%s/%s' % (DATA_DIR, POISONED_FILE)))
X_ps=X_ps.reshape(X_ps.shape[0],X_ps.shape[2],X_ps.shape[3],X_ps.shape[1])
preds=[]
i=0
for x in X_ps:
    print(i)
    i=i+1
    #If poisoned set output variable as N+1th class
    if detect_adversial_inputs(model,x,mean_act):
         preds.append(totalClasses)
    else:
         p=model.predict(x)
         preds.append(p)
accuracy = np.mean(np.equal(preds, Y_ps))
